In [1]:
import gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import feather
import datetime
from matplotlib_venn import venn2
from sklearn import preprocessing
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import LatentDirichletAllocation as LDA
pd.set_option('display.max_Columns', 100)

In [2]:
cartlog = pd.read_feather('../inputs/cartlog.f')
product_master = pd.read_feather('../inputs/product_master.f')
meta = pd.read_feather('../inputs/meta.f')
user_master = pd.read_feather('../inputs/user_master.f')
test = pd.read_csv('../inputs/test.csv')
display_action_id = pd.read_csv('../inputs/display_action_id.csv')

product_master['JAN'] = product_master['JAN'].astype(str)

In [3]:
test_sessions = test["session_id"].unique()
print(len(test_sessions))
test_input_log = cartlog[cartlog["session_id"].isin(test_sessions)]

56486


In [4]:
target_category = [
    38,  # アイスクリーム__ノベルティー
    110,  # スナック・キャンディー__ガム
    113,  # スナック・キャンディー__シリアル
    114,  # スナック・キャンディー__スナック
    134,  # チョコ・ビスクラ__チョコレート
    171,  # ビール系__RTD
    172,  # ビール系__ノンアルコール
    173,  # ビール系__ビール系
    376,  # 和菓子__米菓
    435,  # 大型PET__無糖茶（大型PET）
    467,  # 小型PET__コーヒー（小型PET）
    537,  # 水・炭酸水__大型PET（炭酸水）
    539,  # 水・炭酸水__小型PET（炭酸水）
    629,  # 缶飲料__コーヒー（缶）
    768,  # 麺類__カップ麺
]

In [5]:
# 2020-08-01以前で10分以上経過し購買が発生したセッションにtrainデータを絞る
tmp_sessions = meta[(meta['date'] < '2020-08-01')]['session_id'].unique()
tmp_log = cartlog[cartlog["session_id"].isin(tmp_sessions)]
print('2020-08-01以前: ', len(tmp_sessions))

# 購買が発生したセッション
payment_sessions = set(tmp_log[tmp_log['is_payment']==1]['session_id'].unique())
print('購買が発生: ', len(payment_sessions))
# 10分以上のセッション
over10min_sessions = set(tmp_log[tmp_log['spend_time']>=600]['session_id'].unique())
print('10分以上: ', len(over10min_sessions))
# 積集合
all_train_sessions = payment_sessions & over10min_sessions
print('積集合: ', len(all_train_sessions))

# 10分以上の全trainのログデータ
all_train_log = tmp_log[tmp_log["session_id"].isin(all_train_sessions)]
print('全trainのログ: ', len(all_train_log))

2020-08-01以前:  663708
購買が発生:  618462
10分以上:  404825
積集合:  391383
全trainのログ:  10826062


In [6]:
def agg_payment(cartlog) -> pd.DataFrame:
    """セッションごと・商品ごとの購買個数を集計する"""
    # JANコード (vale_1)ごとに商品の購入個数(n_items)を足し算
    agg = cartlog.loc[cartlog["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    return pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner").drop(columns=['JAN'])

In [7]:
class RetailDataset:
    def __init__(self, thres_sec, meta):
        self.thres_sec = thres_sec
        self.meta = meta.copy()
        self.meta['time_elapsed_sec'] = self.meta['time_elapsed'] * 60
        self.meta.loc[self.meta['time_elapsed_sec'].isnull(), 'time_elapsed_sec'] = thres_sec
        
        # all_train_logの中で、指定時間より前のログのみを抽出　-> public_train_log, train_sessions
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        self.public_train_log = merge_train[merge_train['spend_time'] <= merge_train['time_elapsed_sec']]
        self.train_sessions = self.public_train_log["session_id"].unique()
        
    def get_train_input_log(self) -> pd.DataFrame:
        return self.public_train_log

    def get_train_target(self) -> pd.DataFrame:
        """学習で使用するセッションの目的変数を取得する"""
        train_target = pd.DataFrame(
            index=self.train_sessions,
        )
        train_target.index.name = "session_id"

        # time_elapsed以降のデータから購買個数を集計する
        merge_train = pd.merge(all_train_log, self.meta[["session_id", "time_elapsed_sec"]], on=["session_id"], how="inner")
        after_elapsed_log = merge_train[merge_train['spend_time'] > merge_train['time_elapsed_sec']]
        
        train_item_num = agg_payment(after_elapsed_log)
        train_item_num = train_item_num[train_item_num['category_id'].isin(target_category)]
        train_target_pos = train_item_num.groupby(["session_id", "category_id"])["n_items"].sum().unstack().fillna(0).astype(int)
        train_target_pos[train_target_pos > 0] = 1
        train_target_pos[train_target_pos <= 0] = 0

        return train_target.join(train_target_pos).fillna(0).reset_index()

In [8]:
def get_train_log(elapsed_min):
    dataset = RetailDataset(elapsed_min*60, meta)
    train_input_log = dataset.get_train_input_log()
    y_train = dataset.get_train_target()
    print('train_session', y_train.shape)
    return train_input_log, y_train

In [9]:
ELAPSED_MIN = [0, 3, 5, 10]
# ELAPSED_MIN = [5]

In [10]:
LOG_VER = 5

In [11]:
def save_train_log(ver):
    for elap_min in ELAPSED_MIN:
        train_log, train_y = get_train_log(elap_min)
        train_log = train_log.reset_index(drop=True)
        train_y = train_y.reset_index(drop=True)
        train_y.columns = [str(c) for c in train_y.columns]
        train_log.to_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        train_y.to_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))

In [12]:
def load_train_log(ver):
    train_log = {}
    train_y = {}
    for elap_min in ELAPSED_MIN:
        log = pd.read_feather('../inputs/train{}_log_{}.f'.format(ver, elap_min))
        y = pd.read_feather('../inputs/train{}_y_{}.f'.format(ver, elap_min))
        train_log[elap_min] = log
        train_y[elap_min] = y
    return train_log, train_y

In [13]:
# save_train_log(LOG_VER)

#### ver.5: 2020-04-01以前
- 0, train_session (277032, 16)
- 3, train_session (287301, 16)
- 5, train_session (288227, 16)
- 10, train_session (288664, 16)

#### ver.4: 2020-01-01以降、2020-08-01以前
- 0, train_session (170752, 16)
- 3, train_session (172654, 16)
- 5, train_session (172794, 16)
- 10, train_session (172835, 16)

#### ver.3: 2020-04-01以降、2020-08-01以前
- 0, train_session (101562, 16)
- 3, train_session (102348, 16)
- 5, train_session (102394, 16)
- 10, train_session (102410, 16)

#### ver.2: 2020-08-01以前
- 0, train_session (378594, 16)
- 3, train_session (389649, 16)
- 5, train_session (390621, 16)
- 10, train_session (391074, 16)

### 過去のログデータ

In [14]:
payed_item = agg_payment(all_train_log)

In [15]:
def LDA_topic(df_input, topic, index, prefix):
    df_cp = df_input.set_index(index)
    lda = LDA(n_components=topic)
    lda_out = pd.DataFrame(lda.fit_transform(df_cp), index=df_cp.index).add_prefix(prefix)
    return lda_out.reset_index()

### ユーザ情報

In [16]:
user_features = pd.merge(meta[["session_id", "user_id"]], user_master, on="user_id", how="left")
user_features.loc[user_features['age'] >= 80, 'age'] = np.NaN
user_features.loc[user_features['age'] < 10, 'age'] = np.NaN
user_features.loc[user_features['gender'] > 1, 'gender'] = np.NaN

In [17]:
def get_user_item(payed_item):
    # train用のデータから購買した商品
    user_payed_item = pd.merge(payed_item, meta[['session_id', 'user_id']], on='session_id', how='left')
    group_user_item = user_payed_item.groupby(['user_id', 'category_id'])[['n_items']].sum().reset_index()
    pivot_user_item = group_user_item.pivot_table(index='user_id', columns='category_id', values='n_items')
    # 全ユーザーの購入数の合計が5000以上のカテゴリに絞り込み
    sum_user_item = pivot_user_item.sum()
    user_item_index = sum_user_item[sum_user_item > 5000].index
    pivot_user_item = pivot_user_item[user_item_index].fillna(0).reset_index()
    # trainに存在しない人用に平均値で穴埋め
    user_item_mean = pivot_user_item.mean()
    # 全ユーザーとマージ
    all_user_item = pd.merge(user_master[['user_id']], pivot_user_item, on='user_id', how='left')
    # targetのカテゴリは除く
    for col in all_user_item.columns:
        if (col == 'user_id') or (col in target_category):
            continue
        new_col = 'user_pay_{}'.format(col)
        all_user_item[new_col] = all_user_item[col].fillna(user_item_mean[col]).astype('float32')
        all_user_item.loc[all_user_item[new_col]<0, new_col] = 0

    return all_user_item.drop(columns=list(user_item_mean.index))

In [18]:
all_user_item = get_user_item(payed_item)
print(all_user_item.shape)

(40350, 231)


In [19]:
def save_item_lda(ver, all_user_item):
    user_lda = LDA_topic(all_user_item, 10, 'user_id', 'LDA_')
    user_lda.to_feather('../inputs/user_lda_{}.f'.format(ver))

In [20]:
def load_item_lda(ver):
    user_lda = pd.read_feather('../inputs/user_lda_{}.f'.format(ver))
    return user_lda

In [21]:
# %%time
# save_item_lda(LOG_VER, all_user_item)

In [22]:
user_lda = load_item_lda(LOG_VER)
all_user_item = pd.merge(all_user_item, user_lda, on='user_id', how='left')
print(all_user_item.shape)

(40350, 241)


### メタ情報

In [23]:
def get_meta_features(meta):
    meta_features = meta.copy()
    meta_features['year'] = meta_features['date'].dt.year
    meta_features['month'] = meta_features['date'].dt.month
    meta_features['day'] = meta_features['date'].dt.day
    meta_features['dow'] = meta_features['date'].dt.dayofweek
#     meta_features['doy'] = meta_features['date'].dt.dayofyear
    meta_features['week_time'] = meta_features['dow'] * 24 + meta_features['hour']
    le = preprocessing.LabelEncoder()
    meta_features['userid'] = le.fit_transform(meta_features['user_id'])
    
    # 曜日x時間の来店者数
    df_tz = meta_features.groupby(['week_time']).size().rename('timezone_count')
    df_tz =  pd.DataFrame(df_tz).reset_index()
    meta_features = pd.merge(meta_features, df_tz, on='week_time', how='left')
    
    # userごとに前の来店からどれくらいの日数が経ったか
    df = meta_features.groupby(['session_id', 'user_id', 'date']).first().reset_index().sort_values(['user_id', 'session_id'])
    df['date_diff'] = df['date'].diff(1)
    df['user_diff'] = df['user_id'].shift(1)
    df.loc[df['user_diff'] != df['user_id'], 'date_diff'] = pd.NaT
    df['date_diff'] = df['date_diff'].dt.days
    meta_features = meta_features.join(df[['date_diff']])

    # userごとの来店回数
    meta_features['date_rank'] = meta_features.groupby(['user_id'])['date'].rank(ascending=True)
    
    return meta_features.drop(columns=['user_id', 'date', 'time_elapsed', 'date_str'])

In [24]:
meta_features = get_meta_features(meta)

### ディスプレイアクション

In [25]:
disp_name_dic = {}
for i, disp in enumerate(display_action_id['display_name'].unique()):
    disp_name_dic[disp] = 'disp_cnt_{}'.format(i)
    
act_name_dic = {}
for i, action in enumerate(display_action_id['action_name'].unique()):
    act_name_dic[action] = 'act_cnt_{}'.format(i)

In [26]:
def get_display_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    disp_group_count = merge.groupby(['session_id', 'display_name']).size().reset_index().rename(columns={0:'disp_name_count'})
    disp_name_pivot = disp_group_count.pivot_table(index='session_id', columns='display_name', values='disp_name_count', aggfunc='sum')
    disp_name_pivot = disp_name_pivot.reset_index().fillna(0).rename(columns=disp_name_dic)
    
    disp_out = disp_name_pivot[['session_id']].copy()
    for val in disp_name_dic.values():
        disp_out[val] = 0
    
    for col in disp_name_pivot.columns:
        if col == 'session_id':
            continue
        disp_out[col] = disp_name_pivot[col]
    
    return disp_out

In [27]:
def get_action_name_feature(input_log):
    merge = pd.merge(input_log, display_action_id, on='display_action_id', how='left')
    act_group_count = merge.groupby(['session_id', 'action_name']).size().reset_index().rename(columns={0:'act_name_count'})
    act_name_pivot = act_group_count.pivot_table(index='session_id', columns='action_name', values='act_name_count', aggfunc='sum')
    act_name_pivot = act_name_pivot.reset_index().fillna(0).rename(columns=act_name_dic)
    
    act_out = act_name_pivot[['session_id']].copy()
    for val in act_name_dic.values():
        act_out[val] = 0
    
    for col in act_name_pivot.columns:
        if col == 'session_id':
            continue
        act_out[col] = act_name_pivot[col]
    
    return act_out

### セッション単位の特徴量

In [28]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'value_1']].rename(columns={'value_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, lda_coupon, on='coupon', how='left').drop(columns=['coupon'])
#     session_coupon = session_coupon.groupby(['session_id']).max().reset_index()
#     return session_coupon

In [29]:
# def get_coupon_info(input_log):
#     session_coupon = input_log[input_log["kind_1"] == "クーポン"][['session_id', 'name_1']].rename(columns = {'name_1':'coupon'})
#     session_coupon = pd.merge(session_coupon, df_coupon_cat, on='coupon', how='left').drop(columns=['coupon'])
    
#     for cat in df_coupon_cat['coup_cat'].unique():
#         new_col = 'coup_cat_{}'.format(cat)
#         session_coupon[new_col] = 0
#         session_coupon.loc[session_coupon['coup_cat'] == cat, new_col] = 1
    
#     session_coupon.drop(columns=['coup_cat'], inplace=True)
#     return session_coupon.groupby('session_id').sum().reset_index()

In [30]:
def get_pre_payment_item(input_log):
    session_unique = input_log['session_id'].unique()
    agg = input_log.loc[input_log["kind_1"] == "商品"].groupby(["session_id", "value_1"])["n_items"].sum().reset_index()
    agg = agg.rename(columns={"value_1": "JAN"})
    agg = pd.merge(agg, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    agg = agg[agg['category_id'].isin(target_category)]
    agg = agg.groupby(["session_id", "category_id"])["n_items"].sum().reset_index()
    
    sesi = np.zeros(len(target_category))
    cate = [ct for ct in target_category]
    
    dummy = pd.DataFrame({'session_id':sesi, 'category_id':cate, 'n_items':sesi})
    agg = pd.concat([agg, dummy])
    
    agg = agg.pivot_table(index='session_id', columns='category_id', values='n_items').fillna(0)
    src_columns = ['x_{}'.format(c) for c in agg.columns]
    agg.columns = src_columns
    
    col = ['pre_target_{}'.format(c) for c in target_category]
    df_out = pd.DataFrame(index=session_unique, columns=col)
    df_out.index.name = "session_id"
    df_out = df_out.join(agg)
    for ct in target_category:
        src = 'x_{}'.format(ct)
        dst = 'pre_target_{}'.format(ct)
        df_out[dst] = df_out[src]
    
    return df_out.drop(columns=src_columns).fillna(0).reset_index()

In [31]:
def get_session_kind_group(input_log):
    kind_name ={
        'クーポン': 'coupon',
        '会計': 'kaikei',
        'キー': 'key',
        'カテゴリ': 'categry',
        'バーコードスキャン': 'barcode',
        'UUID': 'uuid',
        '使用ポイント': 'usedpoint',
        '確認': 'confirm',
        'ブランドスイッチ': 'bland',
        'レシピ': 'recipe',
        'スマホスキャン': 'smartphone',
        '磁気スキャン': 'magnetic',
        'レコメンド': 'recommend',
        '倍率ポイント': 'point',
    }
    group_count = input_log[input_log['kind_1'] == '商品'].groupby(["session_id"]).size().rename('group_count_'+'item')
    for kind, name in kind_name.items():
        tmp = input_log[input_log['kind_1'] == kind].groupby(["session_id"]).size().rename('group_count_'+name)
        group_count = pd.concat([group_count, tmp], axis=1)
        
    return group_count.reset_index()

In [32]:
def get_session_item_info(input_log):
    item_log = input_log[input_log['kind_1'] == '商品'].copy()
    item_log = item_log.rename(columns={"value_1": "JAN"})
    item_log = pd.merge(item_log, product_master[["JAN", "category_id"]], on="JAN", how="inner")
    item_log['total'] = item_log['n_items'] * item_log['unit_price']
    session_item = item_log.groupby(['session_id']).agg({
        'total':'sum', 
        'number_1':'sum', 
        'n_items':'sum', 
        'name_1':'nunique',
        'category_id':'nunique',
    }).reset_index()
    session_item = session_item.rename(columns={
        'number_1':'cart_item_cnt', 
        'n_items':'total_item_cnt', 
        'name_1':'item_nunique',
        'category_id':'cat_nunique',
    })
    session_item['mean_price'] = session_item['total'] / session_item['total_item_cnt']
    session_item['item_cnt_per_nuniq'] = session_item['total_item_cnt'] / session_item['item_nunique']
    session_item['item_nuniq_per_cat_nuniq'] = session_item['item_nunique'] / session_item['cat_nunique']
    session_item['mean_price_per_cat'] = session_item['total'] / session_item['cat_nunique']
    return session_item

In [33]:
def get_session_info(input_log):
    # アクション数
    n_actions = input_log.groupby(["session_id"]).size().rename("n_actions")
    # 経過時間の平均
    mean_spend_time = input_log.groupby(["session_id"])["spend_time"].mean()
    # ユニークユーザー
    unique_user = input_log.groupby(["session_id"])['user_id'].nunique().rename("uniq_user")
    
    session_features = pd.concat([
        n_actions,
        mean_spend_time,
        unique_user,
    ], axis=1)
    
    session_features['n_actions_user'] = session_features['n_actions'] * session_features['uniq_user']
    session_features['spend_time_user'] = session_features['spend_time'] * session_features['uniq_user']
    
    return session_features.reset_index()

### セッション単位で集計

In [34]:
def get_session_features(input_log):
    df_ses = pd.DataFrame(columns=['session_id'])
    
    session_feat = [
        get_session_info(input_log),
        get_session_kind_group(input_log),
        get_display_name_feature(input_log),
        get_action_name_feature(input_log),
        get_pre_payment_item(input_log),
#         get_coupon_info(input_log),
        get_session_item_info(input_log),
    ]
    
    for feat in session_feat:
        df_ses = pd.merge(df_ses, feat, on='session_id', how='outer')
        
    return df_ses

### 特徴量を集約する

In [35]:
def merge_features(input_log, session):
    feat_list = [
        get_session_features(input_log),
        user_features,
        meta_features,
    ]
    out = pd.DataFrame({"session_id": session})
    for feat in feat_list:
        out = pd.merge(out, feat, on="session_id", how="left")
        
    # userの情報
    out = pd.merge(out, all_user_item, on='user_id', how='left').drop(columns='user_id')

    assert len(session) == len(out)
    return out

In [36]:
def get_train_all_features(elapsed_min, train_log_list, train_y_list):
    train_input_log = train_log_list[elapsed_min]
    y_train = train_y_list[elapsed_min]
    
    train_features = merge_features(train_input_log, y_train['session_id'])
    print('train_features', train_features.shape)
    return train_features, y_train

In [37]:
def get_test_all_feature(elapsed_min):
    test_meta = meta[meta['session_id'].isin(test_sessions)]
    test_meta = test_meta[test_meta['time_elapsed'] == elapsed_min]
    test_input_elapsed = pd.merge(test_input_log, test_meta[['session_id']], on='session_id', how='left')
    
    test_features = merge_features(test_input_elapsed, test_meta['session_id'])
    print('test_features', test_features.shape)
    return test_features

In [38]:
n_fold = 4

In [39]:
lgbm_param = {
    'objective' : 'binary',
    'boosting_type': 'gbdt',
    'metric': 'auc',
    'seed' : 0,
    'learning_rate':  0.1,
#   'max_depth': 6,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': -1
}

In [40]:
def train_lgbm(X, y, params=lgbm_param):

    fold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=0)
    cv = fold.split(X, y)
    
    models = []
    oof_pred = np.zeros_like(y, dtype=np.float)
    
    cat_feat = ['age', 'gender', 'dow', 'register_number', 'userid']

    for i, (idx_train, idx_valid) in enumerate(cv): 
        x_train, y_train = X.iloc[idx_train], y.iloc[idx_train]
        x_valid, y_valid = X.iloc[idx_valid], y.iloc[idx_valid]
        
        lgbm_train = lgbm.Dataset(x_train, y_train, categorical_feature = cat_feat)
        lgbm_eval = lgbm.Dataset(x_valid, y_valid, reference=lgbm_train, categorical_feature = cat_feat)
        
        lgbm_model = lgbm.train(params, 
                                                    lgbm_train, 
                                                    valid_sets=lgbm_eval,
                                                    categorical_feature = cat_feat,
                                                    num_boost_round=10000,
                                                    early_stopping_rounds=100,
                                                    verbose_eval=-1)
        y_pred = lgbm_model.predict(x_valid, num_iteration=lgbm_model.best_iteration)
        
        oof_pred[idx_valid] = y_pred
        models.append(lgbm_model)

    score = roc_auc_score(y, oof_pred)
    print('--- FINISHED \ whole score: {:.4f} ---'.format(score))
    return oof_pred, models, score

In [41]:
def predict(models, feature):
    pred_list = []
    for i, model in enumerate(models):
        pred = model.predict(feature, num_iteration = model.best_iteration)
        pred_list.append(pred)
    
    score = np.mean(pred_list, axis=0)
    return score

In [42]:
train_log_list, train_y_list = load_train_log(LOG_VER)

In [43]:
gc.collect()

60

In [44]:
%%time
df_pred_all = pd.DataFrame()
df_score_all = pd.DataFrame(index=ELAPSED_MIN)
models_list_list = []

for elapsed_min in ELAPSED_MIN:
    print(f'===== {elapsed_min} =====')
    train_features, y_train = get_train_all_features(elapsed_min, train_log_list, train_y_list)
    test_features = get_test_all_feature(elapsed_min)

    df_pred = pd.DataFrame(index=test_features['session_id'])
    train_features.drop(columns=['session_id'], inplace=True)
    test_features.drop(columns=['session_id'], inplace=True)
    
    models_list = []
    for target in y_train.columns:
        if target == 'session_id':
            continue
        
        print(f"---- id = {target} -----")
        oof, models, score = train_lgbm(train_features, y_train[target])
        models_list.append(models)

        pred = predict(models, test_features)
        df_pred[target] = pred
        df_score_all.loc[elapsed_min, target] = score
        
    models_list_list.append(models_list)
    df_pred_all = pd.concat([df_pred_all, df_pred])
    print(len(df_pred_all))

===== 0 =====
train_features (277032, 344)
test_features (14277, 344)
---- id = 38 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[423]	valid_0's auc: 0.821995
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[176]	valid_0's auc: 0.817686
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[331]	valid_0's auc: 0.815933
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[421]	valid_0's auc: 0.818551
--- FINISHED \ whole score: 0.8183 ---
---- id = 110 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[20]	valid_0's auc: 0.778927
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[19]	valid_0's auc: 0.768225
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[120]	valid_0's auc: 0.783579
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[312]	valid_0's auc: 0.771692
--- FINISHED \ whole score: 0.7545 ---
---- id = 113 -----


...

Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[385]	valid_0's auc: 0.83857
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[158]	valid_0's auc: 0.821868
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[116]	valid_0's auc: 0.828065
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[154]	valid_0's auc: 0.837372
--- FINISHED \ whole score: 0.8276 ---
---- id = 629 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[113]	valid_0's auc: 0.870206
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[70]	valid_0's auc: 0.87601
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[79]	valid_0's auc: 0.871823
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[199]	valid_0's auc: 0.86508
--- FINISHED \ whole score: 0.8598 ---
---- id = 768 -----


/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/home/umhrysnb/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[77]	valid_0's auc: 0.714142
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[113]	valid_0's auc: 0.722664
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[68]	valid_0's auc: 0.717557
Training until validation scores don't improve for 100 rounds
Early stopping, best iteration is:
[179]	valid_0's auc: 0.718385
--- FINISHED \ whole score: 0.7176 ---
56486
CPU times: user 20h 34min 34s, sys: 11h 2min 25s, total: 1d 7h 36min 59s
Wall time: 1h 28min 8s


In [45]:
df_pred_all.head()

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
session_id,,,,,,,,,,,,,,,
663721,0.022094,0.017256,0.006968,0.433539,0.092605,0.064784,0.000147,0.040799,0.098121,0.025612,0.014963,0.001167,0.017866,0.242570,0.017724
663761,0.183820,0.167044,0.420704,0.494310,0.209796,0.063301,0.087549,0.011858,0.234916,0.002645,0.001227,0.000669,0.014376,0.000802,0.341114
663763,0.047679,0.049176,0.162632,0.884380,0.581517,0.011932,0.000809,0.012880,0.162785,0.023672,0.005722,0.000229,0.003669,0.001506,0.057271
663775,0.015309,0.005421,0.003380,0.147094,0.398766,0.130271,0.000203,0.770613,0.026212,0.094596,0.004798,0.000198,0.002864,0.001079,0.157336
663778,0.036273,0.007460,0.011910,0.249628,0.094925,0.040325,0.009838,0.074439,0.064948,0.051941,0.004088,0.017984,0.019112,0.005585,0.095737


In [46]:
df_score_all

,38,110,113,114,134,171,172,173,376,435,467,537,539,629,768
0,0.818294,0.754483,0.797713,0.749641,0.736108,0.859208,0.859620,0.853706,0.743648,0.850105,0.859532,0.898329,0.848381,0.898434,0.735521
3,0.819189,0.772535,0.791797,0.747837,0.734249,0.852836,0.842598,0.848882,0.744416,0.838356,0.842648,0.893211,0.842582,0.898554,0.728125
5,0.819664,0.757748,0.787468,0.742656,0.731130,0.847904,0.832920,0.842814,0.742718,0.830270,0.830544,0.879405,0.831986,0.891949,0.723032
10,0.816109,0.742208,0.777925,0.746236,0.735604,0.832840,0.823511,0.835258,0.742665,0.819058,0.818452,0.865822,0.827564,0.859811,0.717613


In [47]:
cv = df_score_all.mean(axis=1)
print(cv)
print('- cv =', cv.mean())

0     0.817515
3     0.813188
5     0.806147
10    0.797379
dtype: float64
- cv = 0.8085571067821786


In [48]:
assert len(df_pred_all) == len(test)

In [49]:
submission = pd.merge(test[['session_id']], df_pred_all.reset_index(), on='session_id', how='inner')
assert len(submission) == len(test)

In [50]:
submission.drop(columns='session_id').to_csv('../outputs/submission.csv', index=False)

#### baseline_18: (ver.5) 2020-04-01以前
- feat = 344
- Wall time: 1h 28min 8s
- cv = 0.80855 (0.817/0.817/0.817/0.817)
- LB = 0.7107

#### baseline_17: (ver.4) LDAなし
- feat = 256
- Wall time: 55min 11s
- cv = 0.78172 (0.801/0.785/0.776/0.763)
- LB = 0.7324

#### baseline_16: (ver.4) LDAのみ
- feat = 114
- Wall time: 40min 11s
- cv = 0.73996 (0.767/0.741/0.730/0.719)
- LB = 0.695

#### baseline_15: (ver.4) 2020-01-01以降, doyを除外, register_numberをcategoryに
- feat = 266
- Wall time: 53min 34s
- cv = 0.78354 (0.800/0.788/0.779/0.766)
- LB = 0.7333

#### baseline_14: (ver.4) 2020-01-01以降
- feat = 267
- Wall time: 36min 38s
- cv = 0.80381 (0.819/0.807/0.799/0.788)
- LB = 0.7473

#### baseline_13: (ver.3) 2020-04-01以降
- feat = 208
- Wall time: 25min 3s
- cv = 0.78581 (0.803/0.788/0.779/0.772)
- LB = 0.7304

#### baseline_12: (ver.2) 特徴量削減, lr=0.05
- feat = 345
- Wall time: 2h 18min 19s
- cv = 0.82506 (0.835/0.826/0.821/0.816)
- LB = 0.7675

#### baseline_11: クーポンカテゴリの種類減らす, ユニークユーザー数, 平均単価, ユニーク商品数 
- feat = 375
- Wall time: 2h 18min 19s
- cv = 0.81843 (0.831/0.820/0.814/0.806)
- LB = 0.7626

#### baseline_10: クーポンカテゴリの表示回数
- feat = 412
- Wall time: 2h 8min 53s
- cv = 0.821003 (0.831/0.823/0.818/0.8103)
- LB = 0.7657

#### baseline_9-1: 'feature_fraction': 0.6,'bagging_fraction': 0.6,'bagging_freq': 2,
- feat = 333
- Wall time: 2h 2min 14s
- cv = 0.81961 (0.829/0.822/0.816/0.809)
- LB = 0.7658

#### baseline_9: 来店間隔、回数、曜日x時間の来店者数
- feat = 333
- Wall time: 2h 9min 51s
- cv = 0.82118 (0.831/0.824/0.818/0.810)
- LB = 0.7665

#### baseline_8: クーポン発行とターゲットの購買実績
- feat = 344
- Wall time: 2h 5min 13s
- cv = 0.820398 (0.832/0.822/0.817/0.809)
- LB = 0.7484

#### baseline_7: userごとのカテゴリの過去の購買実績
- feat = 329
- Wall time: 1h 59min 16s
- cv = 0.82148 (0.832/0.824/0.818/0.810)
- LB = 0.7671

#### baseline_6: sesssion内のターゲットの購買
- feat = 89
- Wall time: 37min 47s
- cv = 0.66015
- LB = 0.6247

#### baseline_5
- feat = 74
- Wall time: 39min 18s
- cv = 0.66859
- LB = 0.6230

In [51]:
def feat_imp(model):
    fi = model.feature_importance()
    fn = model.feature_name()
    df_feature_importance = pd.DataFrame({'name':fn, 'imp':fi})
    df_feature_importance.sort_values('imp', inplace=True)
    return df_feature_importance

def feature_importance(models):
    fi = pd.DataFrame(columns=['name'])
    for i, model in enumerate(models):
        fi_tmp = feat_imp(model)
        colname = 'imp_{}'.format(i)
        fi_tmp.rename(columns={'imp': colname}, inplace=True)
        fi = pd.merge(fi, fi_tmp, on=['name'], how='outer')
    fi['sum'] = fi.sum(axis=1)
    return fi.sort_values(['sum'], ascending=False)

In [52]:
models = models_list_list[2]

In [53]:
feature_importance(models[0]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,3342,2052,3501,4206,13101
341,register_number,668,464,658,739,2529
340,month,108,100,127,127,462
339,date_rank,99,58,112,112,381
337,date_diff,88,61,92,118,359
338,mean_price_per_cat,91,41,92,116,340
336,hour,79,52,80,76,287
334,week_time,73,41,65,82,261
335,mean_price,77,29,60,91,257
331,LDA_2,58,38,72,66,234


In [54]:
feature_importance(models[1]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,410,285,165,242,1102
341,register_number,49,35,17,27,128
340,date_diff,25,22,10,13,70
339,date_rank,21,18,12,16,67
334,LDA_6,13,13,10,9,45
338,mean_price_per_cat,17,8,6,8,39
333,LDA_2,11,10,6,10,37
337,mean_price,16,6,5,7,34
336,week_time,14,8,3,9,34
326,spend_time_user,9,9,4,7,29


In [55]:
feature_importance(models[2]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,1996,2797,1281,1933,8007
341,register_number,360,526,188,357,1431
340,date_rank,107,139,59,102,407
338,date_diff,71,106,54,71,302
339,mean_price,81,108,40,71,300
337,LDA_2,63,84,35,55,237
336,total,53,80,22,58,213
334,mean_price_per_cat,46,94,22,50,212
335,spend_time,48,91,21,36,196
332,week_time,44,74,21,28,167


In [56]:
feature_importance(models[3]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,2170,3657,3457,3364,12648
341,register_number,448,731,702,717,2598
338,date_rank,53,95,109,87,344
340,date_diff,65,95,83,80,323
339,user_pay_135,59,61,50,60,230
337,LDA_6,43,58,56,51,208
333,mean_price_per_cat,28,69,42,51,190
328,spend_time,21,46,58,50,175
332,mean_price,25,51,50,46,172
335,month,31,48,46,42,167


In [57]:
feature_importance(models[4]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,1421,2496,1384,1472,6773
341,register_number,247,540,231,260,1278
340,user_pay_135,69,71,69,58,267
339,date_rank,50,75,54,51,230
338,month,47,69,49,56,221
337,date_diff,37,53,36,35,161
336,LDA_6,36,42,31,37,146
335,LDA_8,24,37,18,18,97
333,mean_price,19,38,18,15,90
334,mean_price_per_cat,22,35,11,18,86


In [58]:
feature_importance(models[5]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,4284,4794,12544,10627,32249
341,register_number,875,880,1932,1708,5395
340,date_rank,221,231,530,502,1484
339,date_diff,132,126,414,312,984
337,mean_price_per_cat,94,126,384,349,953
338,mean_price,96,94,335,328,853
329,total,58,86,334,316,794
336,spend_time,93,94,264,261,712
333,LDA_2,73,90,253,251,667
335,week_time,80,84,260,223,647


In [59]:
feature_importance(models[6]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,2039,2142,855,6956,11992
341,register_number,350,396,116,1483,2345
340,date_rank,137,151,45,577,910
338,date_diff,96,116,40,444,696
339,mean_price,109,104,27,428,668
337,mean_price_per_cat,94,80,27,416,617
336,total,83,75,17,382,557
334,spend_time,69,58,17,370,514
335,LDA_2,76,84,41,261,462
330,spend_time_user,48,66,16,303,433


In [60]:
feature_importance(models[7]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,6978,3910,3241,7811,21940
341,register_number,1213,790,747,1330,4080
340,date_rank,239,122,103,297,761
339,date_diff,170,95,85,179,529
337,mean_price_per_cat,150,86,48,204,488
338,mean_price,156,70,56,172,454
336,LDA_2,133,88,70,148,439
335,week_time,117,72,54,152,395
330,month,100,81,58,134,373
331,total,101,58,52,145,356


In [61]:
feature_importance(models[8]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,1474,1452,1082,1286,5294
341,register_number,213,217,186,225,841
340,date_diff,51,55,41,49,196
339,user_pay_135,47,51,47,47,192
338,date_rank,47,51,38,51,187
337,LDA_6,26,29,29,22,106
333,user_pay_374,22,23,23,27,95
334,LDA_8,23,17,23,27,90
331,LDA_2,22,32,9,24,87
336,mean_price,25,15,13,22,75


In [62]:
feature_importance(models[9]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,5185,13896,3300,5172,27553
341,register_number,927,2148,614,882,4571
340,date_rank,223,741,132,236,1332
338,date_diff,165,630,129,196,1120
337,mean_price,158,590,101,180,1029
339,mean_price_per_cat,187,553,95,164,999
335,total,143,461,99,139,842
336,LDA_2,153,421,104,146,824
333,spend_time,128,446,95,146,815
334,week_time,129,380,75,121,705


In [63]:
feature_importance(models[10]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,2470,1235,1154,8471,13330
341,register_number,462,195,179,1720,2556
340,date_rank,188,77,64,643,972
339,mean_price,118,58,42,547,765
337,mean_price_per_cat,113,37,37,542,729
338,date_diff,118,45,39,517,719
336,total,87,32,43,508,670
335,spend_time,86,42,36,419,583
331,week_time,69,40,46,375,530
332,day,73,23,10,410,516


In [64]:
feature_importance(models[11]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,7440,7491,2213,2711,19855
341,register_number,1600,1623,436,545,4204
340,date_rank,624,627,145,205,1601
339,date_diff,452,435,106,138,1131
338,mean_price,439,423,97,115,1074
337,mean_price_per_cat,412,424,75,101,1012
336,total,405,397,64,105,971
334,spend_time,303,327,75,84,789
335,spend_time_user,323,310,58,71,762
333,week_time,303,314,69,74,760


In [65]:
feature_importance(models[12]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,11551,5471,3570,2483,23075
341,register_number,1954,969,698,457,4078
340,date_rank,684,307,192,136,1319
338,date_diff,531,215,144,98,988
339,mean_price,542,214,126,74,956
336,mean_price_per_cat,412,206,127,76,821
337,spend_time,417,173,100,85,775
335,total,394,148,106,63,711
334,timezone_count,371,164,103,64,702
333,LDA_2,369,157,87,81,694


In [66]:
feature_importance(models[13]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,274,931,533,702,2440
341,register_number,51,175,117,132,475
340,date_rank,50,94,51,62,257
335,mean_price,13,71,30,56,170
334,total,12,55,43,41,151
338,date_diff,17,63,23,40,143
339,mean_price_per_cat,21,48,29,34,132
337,LDA_2,17,38,25,35,115
331,week_time,11,44,30,29,114
298,spend_time,4,45,18,40,107


In [67]:
feature_importance(models[14]).head(10)

,name,imp_0,imp_1,imp_2,imp_3,sum
342,userid,3666,3500,2546,1632,11344
341,register_number,656,618,448,259,1981
340,date_diff,99,95,53,42,289
339,date_rank,85,93,54,31,263
338,month,82,69,52,43,246
334,hour,66,58,51,29,204
337,day,78,64,34,18,194
336,mean_price,69,79,35,11,194
335,mean_price_per_cat,68,73,34,17,192
330,LDA_2,55,59,31,27,172
